# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.8.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

spark.version

Creating ngrok tunnel to allow Spark UI (Optional)
**Only 20 connections/minute!!!**

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Structured Streaming with Apache Kafka

## Example 1

Reading a Kafka topic in AWS.
Before executing this code, replace `kafka:9092` by the right bootstrap server

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-3-91-30-167.compute-1.amazonaws.com:9092") \
  .option("subscribe", "twitter") \
  .load()
  
schema = StructType(
    [
        StructField('id', StringType(), True),
        StructField('timestamp_ms', StringType(), True),
        StructField('user', StringType(), True),
        StructField('geo', StringType(), True),
        StructField('play', StringType(), True),
        StructField('text', StringType(), True)
    ]
)
df.printSchema()

dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

In [ ]:
dataset.writeStream \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("tweets_topic") \
 .start()

In [ ]:
spark.sql("select id, text from tweets_topic").show(10, False)

## Exercise 1

Apply a sliding window each minute, 5 minutes of duration, grouping by id

---



## Exercise 2

Each minute, get the number of tweets received in last 5 minutes

---



## Exercise 3

Get tweets containing the word `Trump` in 1 minute slots

---

